# データの読み込み

In [5]:
from tensorflow.python.keras.datasets import mnist

In [6]:
(x_train, _), (x_test, _)  = mnist.load_data()

In [9]:
#CNNで扱いやすい形に変換
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [10]:
#画像を0~1に正規化
x_train = x_train/255
x_test = x_test/255

# 擬似的なノイズデータの作成

## マスキングノイズデータ

In [12]:
import numpy as np
def make_masking_noise_data(data_x, percent=0.1):
    size = data_x.shape
    masking = np.random.binomial(n=1, p=percent, size=size)
    return data_x*masking

x_train_masked = make_masking_noise_data(x_train)
x_test_masked = make_masking_noise_data(x_test)

## ガウシアンノイズデータ

In [14]:
def make_gaussian_noise_data(data_x, scale=0.8):
    gaussian_data_x = data_x + np.random.normal(loc =0, scale=scale, size=data_x.shape)
    gaussian_data_x = np.clip(gaussian_data_x, 0, 1)
    return gaussian_data_x

x_train_gauss = make_gaussian_noise_data(x_train)
x_test_gauss = make_gaussian_noise_data(x_test)

In [17]:
from IPython.display import display_png
from keras.preprocessing.image import array_to_img

display_png(array_to_img(x_train[0]))
display_png(array_to_img(x_train_gauss[0]))
display_png(array_to_img(x_train_masked[0]))

# CAEモデルの構築

In [26]:
from tensorflow.python.keras.models import  Sequential
from tensorflow.python.keras.layers import Conv2D
from tensorflow.python.keras.layers import  MaxPooling2D

autoencoder = Sequential()

#Encoder箇所
autoencoder.add(
    Conv2D(
        16,
    (3,3),
    1,
    activation='relu',
    padding='same',
    input_shape=(28, 28, 1)
    )
)

autoencoder.add(
    MaxPooling2D(
        (2,2),
        padding='same'
    )
)

autoencoder.add(
    Conv2D(
        8,
        (3,3),
        1,
        activation='relu',
        padding='same'
    )
)

autoencoder.add(
    MaxPooling2D(
        (2,2),
        padding='same'
    )
)

In [28]:
#Decoder箇所
from tensorflow.python.keras.layers import UpSampling2D

autoencoder.add(
    Conv2D(
        8,
        (3,3),
        1,
        activation='relu',
        padding='same'
    )
)

autoencoder.add(UpSampling2D((2,2)))

autoencoder.add(
    Conv2D(
        16,
        (3, 3),
        1,
        activation='relu',
        padding='same',
        )
)

autoencoder.add(UpSampling2D((2, 2)))

autoencoder.add(
    Conv2D(
        1,
        (3, 3),
        1,
        activation='sigmoid',
        padding='same'
    )
)

autoencoder.compile(
    optimizer='adam',
    loss='binary_crossentropy'
)

initial_weight = autoencoder.get_weights()

# モデルのサマリを確認

In [29]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 7, 7, 8)           584       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 8)           584       
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 14, 14, 8)         0         
__________

# ガウシアンノイズデータを用いた学習と予測

In [30]:
autoencoder.fit(
    x_train_gauss,
    x_train,
    epochs=10,
    batch_size=20,
    shuffle=True
)

Epoch 1/10
60000/60000 [==============================] - 65s 1ms/step - loss: 0.1826
Epoch 2/10
60000/60000 [==============================] - 61s 1ms/step - loss: 0.1550
Epoch 3/10
60000/60000 [==============================] - 62s 1ms/step - loss: 0.1503
Epoch 4/10
60000/60000 [==============================] - 60s 995us/step - loss: 0.1481
Epoch 5/10
60000/60000 [==============================] - 63s 1ms/step - loss: 0.1467
Epoch 6/10
60000/60000 [==============================] - 60s 994us/step - loss: 0.1456
Epoch 7/10
60000/60000 [==============================] - 57s 947us/step - loss: 0.1448
Epoch 8/10
60000/60000 [==============================] - 59s 985us/step - loss: 0.1442
Epoch 9/10
60000/60000 [==============================] - 60s 993us/step - loss: 0.1436
Epoch 10/10
60000/60000 [==============================] - 63s 1ms/step - loss: 0.1432


In [31]:
gauss_preds = autoencoder.predict(x_test_gauss)

In [36]:
#ガウシアンノイズ画像
#CAEモデル
#オリジナル画像

for i in range(10):
    display_png(array_to_img(x_test[i]))
    display_png(array_to_img(x_test_gauss[i]))
    display_png(array_to_img(gauss_preds[i]))
    print('-'*25)

-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


# マスキングノイズデータを用いた学習と予測

In [39]:
#CAEモデルの重みを初期化
autoencoder.set_weights(initial_weight)

In [40]:
autoencoder.fit(
    x_train_masked,
    x_train,
    epochs=10,
    batch_size=20,
    shuffle=True
)

Epoch 1/10
60000/60000 [==============================] - 61s 1ms/step - loss: 0.2031
Epoch 2/10
60000/60000 [==============================] - 60s 994us/step - loss: 0.1867
Epoch 3/10
60000/60000 [==============================] - 60s 996us/step - loss: 0.1835
Epoch 4/10
60000/60000 [==============================] - 62s 1ms/step - loss: 0.1815
Epoch 5/10
60000/60000 [==============================] - 62s 1ms/step - loss: 0.1803
Epoch 6/10
60000/60000 [==============================] - 55s 920us/step - loss: 0.1793
Epoch 7/10
60000/60000 [==============================] - 51s 856us/step - loss: 0.1786
Epoch 8/10
60000/60000 [==============================] - 51s 848us/step - loss: 0.1780
Epoch 9/10
60000/60000 [==============================] - 51s 856us/step - loss: 0.1775
Epoch 10/10
60000/60000 [==============================] - 52s 861us/step - loss: 0.1772


In [41]:
masked_preds = autoencoder.predict(x_test_masked)

In [42]:
for i in range(10):
    display_png(array_to_img(x_test[i]))
    display_png(array_to_img(x_test_masked[i]))
    display_png(array_masked_predsmg(masked_preds[i]))
    print('-'*25)

-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------
